In [1]:
# !pip install plotly panel

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

### Read transaction data with categories

In [3]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('transactions_2022_2023_categorized.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

,Date,Name / Description,Expense/Income,Amount (Rupee),Category,Year,Month,Month Name
0,12/30/2023,Raj's Dhaba,Expense,500,NaN,2023,12,Dec
1,12/30/2023,Airtel Bill Payment,Expense,300,NaN,2023,12,Dec
2,12/30/2023,House Rent,Expense,15000,NaN,2023,12,Dec
3,12/30/2023,Sweta's Boutique Purchase,Expense,1200,NaN,2023,12,Dec
4,12/29/2023,Freelance Web Design Project,Income,8000,NaN,2023,12,Dec
5,12/29/2023,Zomato Food Order,Expense,450,NaN,2023,12,Dec
6,12/23/2023,Grocery from D-Mart,Expense,2000,NaN,2023,12,Dec
7,12/22/2023,Tuition Fees Received,Income,6000,NaN,2023,12,Dec
8,12/22/2023,Diwali Donation to NGO,Expense,500,NaN,2023,12,Dec
9,12/20/2023,Salary,Income,25000,NaN,2023,12,Dec


In [4]:
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

### Make pie charts - Income/ Expense breakdown

In [5]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount (Rupee)', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (Rupee)'].sum() 
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (Rupee)'].sum()
    
    if label == 'Expense':
        total_text = "₹" + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "₹ " + str(round(total_income))

    pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig

In [6]:
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
income_pie_fig_2022

### Make bar charts over months in a year

In [7]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (Rupee)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (Rupee)', text_auto='.2s', title=label+" per month", color='Amount (Rupee)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [8]:
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
income_monthly_2022

### Putting all charts together into tabs for 2022/2023

In [9]:
# Pie charts
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
expense_pie_fig_2022 = make_pie_chart(df, 2022, 'Expense')  
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')

# Bar charts
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
expense_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Expense')
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        ('2022', pn.Column(pn.Row(income_pie_fig_2022, expense_pie_fig_2022),
                                                pn.Row(income_monthly_2022, expense_monthly_2022))),
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))
                        )
                )
tabs.show()

C:\Users\v-kausha\AppData\Local\Temp\ipykernel_8488\3507427792.py:17: RuntimeWarning:

invalid value encountered in scalar divide



ValueError: cannot convert float NaN to integer

### Create dashboard

In [ ]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()